In [ ]:
# Импорт Pandas для работы с DataFrame
import pandas as pd

In [ ]:
# Чтение CSV-файла в DataFrame
df = pd.read_csv("Lab3.csv")

# Вывод первых 5 строк для проверки
print(df.head())

               country  child_mort  exports  health  imports  income  \
0          Afghanistan        90.2     10.0    7.58     44.9    1610   
1              Albania        16.6     28.0    6.55     48.6    9930   
2              Algeria        27.3     38.4    4.17     31.4   12900   
3               Angola       119.0     62.3    2.85     42.9    5900   
4  Antigua and Barbuda        10.3     45.5    6.03     58.9   19100   

   inflation  life_expec  total_fer   gdpp  
0       9.44        56.2       5.82    553  
1       4.49        76.3       1.65   4090  
2      16.10        76.5       2.89   4460  
3      22.40        60.1       6.16   3530  
4       1.44        76.8       2.13  12200  


In [ ]:
# удаление ненужного столбца
df = df.drop(columns=['country'])

In [ ]:
# Признаки с инверсией (ниже = хуже → преобразуем в "выше = хуже")
df['life_expec_inv'] = df['life_expec'].max() - df['life_expec']
df['gdpp_inv'] = df['gdpp'].max() - df['gdpp']
df['income_inv'] = df['income'].max() - df['income']
df['health_inv'] = df['health'].max() - df['health']

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df['cluster_kmeans'] = kmeans.fit_predict(df_scaled)

In [ ]:
print(df)

     child_mort  exports  health  imports  income  inflation  life_expec  \
0          90.2     10.0    7.58     44.9    1610       9.44        56.2   
1          16.6     28.0    6.55     48.6    9930       4.49        76.3   
2          27.3     38.4    4.17     31.4   12900      16.10        76.5   
3         119.0     62.3    2.85     42.9    5900      22.40        60.1   
4          10.3     45.5    6.03     58.9   19100       1.44        76.8   
..          ...      ...     ...      ...     ...        ...         ...   
162        29.2     46.6    5.25     52.7    2950       2.62        63.0   
163        17.1     28.5    4.91     17.6   16500      45.90        75.4   
164        23.3     72.0    6.84     80.2    4490      12.10        73.1   
165        56.3     30.0    5.18     34.4    4480      23.60        67.5   
166        83.1     37.0    5.89     30.9    3280      14.00        52.0   

     total_fer   gdpp  life_expec_inv  gdpp_inv  income_inv  health_inv  \
0         5.

In [ ]:
# Средние по кластерам
summary = df.groupby('cluster')[['child_mort', 'gdpp', 'life_expec', 'income', 'total_fer']].mean()
print(summary)